<a href="https://colab.research.google.com/github/NinhDT22022522/Big_Data_Technologies/blob/main/mapreduce_with_bash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mapreduce with bash

In this notebook we're going to use `bash` to write a mapper and a reducer to count words in a file. This example will serve to illustrate the main features of Hadoop's MapReduce framework.

# Table of contents
- [What is MapReduce?](#mapreduce)
- [The mapper](#mapper)
    - [Test the mapper](#testmapper)
- [Hadoop it up](#hadoop)
    - [What is Hadoop Streaming?](#hadoopstreaming)
    - [List your Hadoop directory](#hdfs_ls)
    - [Test MapReduce with a dummy reducer](#dummyreducer)
    - [Shuffling and sorting](#shuffling&sorting)
- [The reducer](#reducer)
    - [Test and run](#run)
- [Run a mapreduce job with more data](#moredata)
    - [Sort the output with `sort`](#sortoutput)
    - [Sort the output with another MapReduce job](#sortoutputMR)
    - [Configure sort with `KeyFieldBasedComparator`](#KeyFieldBasedComparator)
    - [Specifying Configuration Variables with the -D Option](#configuration_variables)
    - [What is word count useful for?](#wordcount)


## What is MapReduce? <a name="mapreduce"></a>

MapReduce is a computing paradigm designed to allow parallel distributed processing of massive amounts of data.

Data is split across several computer nodes, there it is processed by one or more mappers. The results emitted by the mappers are first sorted and then passed to one or more reducers that process and combine the data to return the final result.

![Map & Reduce](https://github.com/groda/big_data/blob/master/mapreduce.png?raw=1)
With [Hadoop Streaming](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html) it is possible to use any programming language to define a mapper and/or a reducer. Here we're going to use the Unix `bash` scripting language ([here](https://www.gnu.org/software/bash/manual/html_node/index.html) is the official documentation for the language).

## The mapper <a name="mapper"></a>
Let's write a mapper script called `map.sh`. The mapper splits each input line into words and for each word it outputs a line containing the word and `1` separated by a tab.

Example: for the input
<html>
<pre>
apple orange
banana apple peach
</pre>
</html>

`map.sh` outputs:
<html>
<pre>
apple   1
orange  1
banana  1
apple  1
peach  1
</pre>
</html>


The <a href="https://ipython.readthedocs.io/en/stable/interactive/magics.html">_cell magic_</a> [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) allows us to write the contents of the cell to a file.

In [1]:
%%writefile map.sh
#!/bin/bash

while read line
do
 for word in $line
 do
  if [ -n "$word" ]
  then
     echo -e ${word}"\t1"
  fi
 done
done

Writing map.sh


After running the cell above, you should have a new file `map.sh` in your current directory.
The file can be seen in the left panel of JupyterLab or by using a list command on the bash command-line.

**Note:** you can execute a single bash command in a Jupyter notebook cell by prepending an exclamation point to the command.

In [2]:
!ls -hl map.sh

-rw-r--r-- 1 root root 126 Apr 24 03:03 map.sh


### Test the mapper <a name="testmapper"></a>
We're going to test the mapper on on the command line with a small text file `fruits.txt` by first creating the text file.
In this file `apple` for instance appears two times, that's what we want our mapreduce job to compute.

In [3]:
%%writefile fruits.txt
apple banana
peach orange peach peach
pineapple peach apple

Writing fruits.txt


In [4]:
!cat fruits.txt

apple banana
peach orange peach peach
pineapple peach apple


Test the mapper

In [5]:
!cat fruits.txt|./map.sh

/bin/bash: line 1: ./map.sh: Permission denied


If the script `map.sh` does not have the executable bit set, you need to set the correct permissions.

In [6]:
!chmod 700 map.sh

## Hadoop it up <a name="hadoop"></a>
Let us now run a MapReduce job with Hadoop Streaming.

### What is Hadoop Streaming <a name="hadoopstreaming"></a>

Hadoop Streaming is a library included in the Hadoop distribution that enables you to develop MapReduce executables in languages other than Java.

Mapper and/or reducer can be any sort of executables that read the input from stdin and emit the output to stdout. By default, input is read line by line and the prefix of a line up to the first tab character is the key; the rest of the line (excluding the tab character) will be the value.

If there is no tab character in the line, then the entire line is considered as key and the value is null. The default input format is specified in the class `TextInputFormat` (see the [API documentation](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/TextInputFormat.html)) but this can can be customized for instance by defining another field separator (see the [Hadoop Streaming documentation](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Customizing_How_Lines_are_Split_into_KeyValue_Pairs).

This is an example of MapReduce streaming invocation syntax:
<html>
<pre>
    mapred streaming \
  -input myInputDirs \
  -output myOutputDir \
  -mapper /bin/cat \
  -reducer /usr/bin/wc

</pre>
</html>

You can find the full official documentation for Hadoop Streaming from Apache Hadoop here: [https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html).

All options for the Hadoop Streaming command are described here: [Streaming Command Options](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#Streaming_Command_Options) and can be listed with the command

In [7]:
!mapred streaming --help

/bin/bash: line 1: mapred: command not found


Now in order to run a mapreduce job that we need to "upload" the input file to the Hadoop file system.

### List your Hadoop directory <a name="hdfs_ls"></a>

With the command `hdfs dfs -l` you can view the content of your HDFS home directory.

`hdfs dfs` you can run a filesystem command on the Hadoop fileystem. The complete list of commands can be found in the [System Shell Guide](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-hdfs/HDFSCommands.html#dfs).

In [8]:
!hdfs dfs -ls

/bin/bash: line 1: hdfs: command not found


Now create a directory `wordcount` with a subdirectory `input` on the Hadoop filesystem.

In [9]:
%%bash
hdfs dfs -mkdir -p wordcount

bash: line 1: hdfs: command not found


CalledProcessError: Command 'b'hdfs dfs -mkdir -p wordcount\n'' returned non-zero exit status 127.

Copy the file fruits.txt to Hadoop in the folder `wordcount/input`.

Why do we need this step? Because the file `fruits.txt` needs to reside on the Hadoop filesystem in order to enjoy of all of the features of Hadoop (data partitioning, distributed processing, fault tolerance).

In [ ]:
%%bash
hdfs dfs -rm -r wordcount/input 2>/dev/null
hdfs dfs -mkdir wordcount/input
hdfs dfs -put fruits.txt wordcount/input

Let's check if the file is there now.

**Note:** it is convenient use the option `-h` for `ls` to show file sizes in human-readable form (showing sizes in Kilobytes, Megabytes, Gigabytes, etc.)

In [ ]:
!hdfs dfs -ls -h -R wordcount/input

### Test MapReduce with a dummy reducer <a name="dummyreducer"></a>

Let's try to run the mapper using a dummy reducer (`/bin/cat` does nothing else than echoing the data it receives).

**Warning:** mapreduce tends to produce a verbose output, so be ready to see a long output. What you should look for is a message of the kind <html><pre>"INFO mapreduce.Job: Job ... completed successfully"</pre></html>

**Note:** at the beginning of next cell you'll see a command `hadoop fs -rmr wordcount/output 2>/dev/null`. This is needed because when you run a job several times mapreduce will give an error if you try to overwrite the same output directory.

In [ ]:
%%bash
hdfs dfs -rm -r wordcount/output 2>/dev/null
mapred streaming \
  -files map.sh \
  -input wordcount/input \
  -output wordcount/output \
  -mapper map.sh \
  -reducer /bin/cat

The output of the mapreduce job is in the `output` subfolder of the input directory. Let's check what's inside it.

In [ ]:
!hdfs dfs -ls wordcount/output

If `output` contains a file named `_SUCCESS` that means that the mapreduce job completed successfully.

**Note:** when dealing with Big Data it's always advisable to pipe the output of `cat` commands to `head` (or `tail`).

In [ ]:
!hdfs dfs -cat wordcount/output/part*|head

We have gotten as expected all the output from the mapper. Something worth of notice is that the data outputted from the mapper _**has been sorted**_. We haven't asked for that but this step is automatically performed by the mapper as soon as the number of reducers is $\gt 0$.

### Shuffling and sorting <a name="shuffling&sorting"></a>
The following picture illustrates the concept of shuffling and sorting that is automatically performed by Hadoop after each map before passing the output to reduce. In the picture the outputs of the two mapper tasks are shown. The arrows represent shuffling and sorting done before delivering the data to one reducer (rightmost box).
![Shuffle & sort](https://github.com/groda/big_data/blob/master/shuffle_sort.png?raw=1)
The shuffling and sorting phase is often one of the most costly in a MapReduce job.


<b>Note:</b> the job ran with two mappers because $2$ is the default number of mappers in Hadoop.

## The reducer <a name="reducer"></a>
Let's now write a reducer script called `reduce.sh`.

In [ ]:
%%writefile reduce.sh
#!/bin/bash

currkey=""
currcount=0
while IFS=$'\t' read -r key val
do
  if [[ $key == $currkey ]]
  then
      currcount=$(( currcount + val ))
  else
    if [ -n "$currkey" ]
    then
      echo -e ${currkey} "\t" ${currcount}
    fi
    currkey=$key
    currcount=1
  fi
done
# last one
echo -e ${currkey} "\t" ${currcount}

Set permission for the reducer script.

In [ ]:
!chmod 700 reduce.sh

### Test and run <a name="run"></a>

Test map and reduce on the shell

In [ ]:
!cat fruits.txt|./map.sh|sort|./reduce.sh

Once we've made sure that the reducer script runs correctly on the shell, we can run it on the cluster.

In [ ]:
%%bash
hdfs dfs -rm -r wordcount/output 2>/dev/null
mapred streaming \
  -file map.sh \
  -file reduce.sh \
  -input wordcount/input \
  -output wordcount/output \
  -mapper map.sh \
  -reducer reduce.sh

Let's check the output on the HDFS filesystem

In [ ]:
!hdfs dfs -cat wordcount/output/part*|head

## Run a mapreduce job with more data <a name="moredata"></a>

Let's create a datafile by downloading some real data, for instance from a Web page. This example will be used to introduce some advanced configurations.

Next, we download a URL with `wget` and filter out HTML tags with a `sed` regular expression.

In [ ]:
%%bash
URL=https://www.derstandard.at/story/2000110819049/und-wo-warst-du-beim-fall-der-mauer
wget -qO- $URL | sed -e 's/<[^>]*>//g;s/^ //g' >sample_article.txt

In [ ]:
!cat sample_article.txt|./map.sh|head

As usual, with real data there's some more work to do. Here we see that the mapper script doesn't skip empty lines. Let's modify it so that empty lines are skipped.

In [ ]:
%%writefile map.sh
#!/bin/bash

while read line
do
 for word in $line
 do
  if [[ "$line" =~ [^[:space:]] ]]
  then
    if [ -n "$word" ]
    then
    echo -e ${word} "\t1"
    fi
  fi
 done
done

In [ ]:
!cat sample_article.txt|./map.sh|head

Now the output of `map.sh` looks better!

<b>Note:</b> when working with real data we need in general some more preprocessing in order to remove control characters or invalid unicode.

Time to run MapReduce again with the new data, but first we need to "put" the data on HDFS.

In [ ]:
%%bash
hdfs dfs -rm -r wordcount/input 2>/dev/null
hdfs dfs -put sample_article.txt wordcount/input

In [ ]:
# check that the folder wordcount/input on HDFS only contains sample_article.txt
!hdfs dfs -ls -h wordcount/input

Check the reducer

In [ ]:
!cat sample_article.txt|./map.sh|./reduce.sh|head

In [ ]:
%%bash
hadoop fs -rmr wordcount/output 2>/dev/null
mapred streaming \
  -file map.sh \
  -file reduce.sh \
  -input wordcount/input \
  -output wordcount/output \
  -mapper map.sh \
  -reducer reduce.sh

Check the output on HDFS

In [ ]:
!hdfs dfs -ls wordcount/output

This job took a few seconds and this is quite some time for such a small file (4KB). This is due to the overhead of distributing the data and running the Hadoop framework.
The advantage of Hadoop can be appreciated only for large datasets.

In [ ]:
!hdfs dfs -cat wordcount/output/part-00000|head

### Sort the output with `sort` <a name="sortoutput"></a>

We've obtained a list of tokens that appear in the file followed by their frequencies.

The output of the reducer is sorted by key (the word) because that's the ordering that the reducer becomes from the mapper. If we're interested in sorting the data by frequency, we can use the Unix `sort` command (with the options `k2`, `n`, `r` respectively "by field 2", "numeric", "reverse").

In [ ]:
!hdfs dfs -cat wordcount/output/part-00000|sort -k2nr|head

The most common word appears to be "die" (the German for the definite article "the").

### Sort the output with another MapReduce job <a name="sortoutputMR"></a>

If we wanted to sort the output of the reducer using the mapreduce framework, we could employ a simple trick: create a mapper that interchanges words with their frequency values. Since by construction mappers sort their output by key, we get the desired sorting as a side-effect.

Call the new mapper `swap_keyval.sh`.

In [ ]:
%%writefile swap_keyval.sh
#!/bin/bash
# This script will read one line at a time and swap key/value
# For instance, the line "word 100" will become "100 word"

while read key val
do
 printf "%s\t%s\n" "$val" "$key"
done

We are going to run the swap mapper script on the output of the previous mapreduce job. Note that in the below cell we are not deleting the previous output but instead we're saving the output from the current job in a new folder `output_sorted`.

Nice thing about running a job on the output of a preceding job is that we do not need to upload files to HDFS because the data is already on HDFS. Not so nice: writing data to disk at each step of a data transformation pipeline takes time and this can be costly for longer data pipelines. This is one of the shortcomings of MapReduce that are addressed by [Apache Spark](https://spark.apache.org/).

In [ ]:
%%bash
hdfs dfs -rm -r wordcount/output2 2>/dev/null
mapred streaming \
  -file swap_keyval.sh \
  -input wordcount/output \
  -output wordcount/output2 \
  -mapper swap_keyval.sh

Check the output on HDFS

In [ ]:
!hdfs dfs -ls wordcount/output2

In [ ]:
!hdfs dfs -cat wordcount/output2/part-00000|head

Mapper uses by default ascending order to sort by key. We could have changed that with an option but for now let's look at the end of the file.

In [ ]:
!hdfs dfs -cat wordcount/output2/part-00000|tail

### Configure sort with `KeyFieldBasedComparator` <a name="KeyFieldBasedComparator"></a>

In general, we can determine how mappers are going to sort their output by configuring the comparator directive to use the special class [`KeyFieldBasedComparator`](https://hadoop.apache.org/docs/current/api/org/apache/hadoop/mapreduce/lib/partition/KeyFieldBasedComparator.html)
<html><pre>-D mapreduce.job.output.key.comparator.class=\
    org.apache.hadoop.mapred.lib.KeyFieldBasedComparator</pre></html>
    
This class has some options similar to the Unix `sort`(`-n` to sort numerically, `-r` for reverse sorting, `-k pos1[,pos2]` for specifying fields to sort by).

Let us see the comparator in action on our data to get the desired result. Note that this time we are removing `output2` because we're running the second mapreduce job again.

In [ ]:
%%bash
hdfs dfs -rmr wordcount/output2 2>/dev/null
comparator_class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator
mapred streaming \
  -D mapreduce.job.output.key.comparator.class=$comparator_class \
  -D mapreduce.partition.keycomparator.options=-nr \
  -file swap_keyval.sh \
  -input wordcount/output \
  -output wordcount/output2 \
  -mapper swap_keyval.sh

In [ ]:
!hdfs dfs -ls wordcount/output2

In [ ]:
!hdfs dfs -cat wordcount/output2/part-00000|head

Now we get the output in the desired order.

### Specifying Configuration Variables with the -D Option <a name="configuration_variables"></a>

With the `-D` option it is possible to override options set in the default configuration file [`mapred_default.xml`](https://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/mapred-default.xml)
(see the [Apache Hadoop documentation](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#Specifying_Configuration_Variables_with_the_-D_Option)).

One option that might come handy when dealing with out-of-memory issues in the sorting phase is the size in MB of the memory reserved for sorting `mapreduce.task.io.sort.mb`:
 <html>
    <pre>-D mapreduce.task.io.sort.mb=512
    </pre>
 </html>

 **Note:** the maximum value for `mapreduce.task.io.sort.mb` is 2047.   

## What is word count useful for? <a name="wordcount"></a>
Counting the frequencies of words is at the basis of _indexing_ and it facilitates the retrieval of relevant documents in search engines.